In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
## connect to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## **Usefull Functions for Analysis**

In [4]:
def merge_imputed_data(tariff_data, missing_data):
    """
    Merges the imputed missing data with the original data and returns the combined DataFrame.

    Parameters:
    tariff_data (pd.DataFrame): The original DataFrame containing the consumption data with potential missing timestamps.
                             It must contain the columns 'LCLid', 'DateTime', and 'Consumption'.
    missing_data (pd.DataFrame): The DataFrame containing the imputed missing data. It must contain the columns
                                 'LCLid', 'MissingDateTime', and 'Consumption' along with 'Hour' and 'Month'.

    Returns:
    pd.DataFrame: A combined DataFrame with the original tariff data and the imputed missing data, sorted by 'LCLid' and 'DateTime'.
    """
    # Append MissingDateTime to original DateTime column in std_data to match the structure
    missing_data['DateTime'] = missing_data['MissingDateTime']
    missing_data = missing_data.drop(columns=['MissingDateTime', 'Hour', 'Month'])

    # Merge the imputed data back to the original std_data
    tariff_data_imputed = pd.concat([tariff_data, missing_data], ignore_index=True)
    tariff_data_imputed = tariff_data_imputed.sort_values(by=['LCLid', 'DateTime']).reset_index(drop=True)

    return tariff_data_imputed

# **Sampling Data taking 50 unique LCLid of customers in STD and ToU Tariff**

In [5]:
import pandas as pd

filename = '/content/drive/MyDrive/LCL-FullData/CC_LCL-FullData.csv'
chunksize = 10**6  # Adjust based on available memory

# Desired date range
start_date = pd.to_datetime('2013-01-01')
end_date = pd.to_datetime('2014-02-28')

# Step 1: Identify IDs with complete date coverage
std_ids = set()
tou_ids = set()

# Read file in chunks
for chunk in pd.read_csv(filename, chunksize=chunksize, parse_dates=['DateTime']):
    # Ensure DateTime column is properly parsed
    chunk['DateTime'] = pd.to_datetime(chunk['DateTime'], errors='coerce')

    # Filter by date range first
    chunk = chunk[(chunk['DateTime'] >= start_date) & (chunk['DateTime'] <= end_date)]

    if len(std_ids) < 50:
        std_chunk = chunk[chunk['stdorToU'] == 'Std']
        grouped_std = std_chunk.groupby('LCLid')
        for name, group in grouped_std:
            if group['DateTime'].min() <= start_date and group['DateTime'].max() >= end_date:
                std_ids.add(name)
            if len(std_ids) >= 50:
                break

    if len(tou_ids) < 50:
        tou_chunk = chunk[chunk['stdorToU'] == 'ToU']
        grouped_tou = tou_chunk.groupby('LCLid')
        for name, group in grouped_tou:
            if group['DateTime'].min() <= start_date and group['DateTime'].max() >= end_date:
                tou_ids.add(name)
            if len(tou_ids) >= 50:
                break

    # Check if we have enough IDs
    if len(std_ids) >= 50 and len(tou_ids) >= 50:
        break

std_ids = list(std_ids)
tou_ids = list(tou_ids)

# Debug: Check if the correct IDs were identified
print(f"STD IDs (count: {len(std_ids)}): {std_ids[:5]}")  # Print first 5 IDs for verification
print(f"TOU IDs (count: {len(tou_ids)}): {tou_ids[:5]}")  # Print first 5 IDs for verification

# Step 2: Collect all rows for selected IDs
std_data = []
tou_data = []

for chunk in pd.read_csv(filename, chunksize=chunksize, parse_dates=['DateTime']):
    chunk['DateTime'] = pd.to_datetime(chunk['DateTime'], errors='coerce')

    std_chunk = chunk[chunk['LCLid'].isin(std_ids) & (chunk['stdorToU'] == 'Std')]
    tou_chunk = chunk[chunk['LCLid'].isin(tou_ids) & (chunk['stdorToU'] == 'ToU')]

    std_data.append(std_chunk)
    tou_data.append(tou_chunk)

# Concatenate the dataframes only if they are not empty
if std_data:
    std_data = pd.concat(std_data)
else:
    std_data = pd.DataFrame()

if tou_data:
    tou_data = pd.concat(tou_data)
else:
    tou_data = pd.DataFrame()

# Filter concatenated data by date range to ensure correctness
std_data = std_data[(std_data['DateTime'] >= start_date) & (std_data['DateTime'] <= end_date)]
tou_data = tou_data[(tou_data['DateTime'] >= start_date) & (tou_data['DateTime'] <= end_date)]

# Debug: Check the range of dates in the resulting data
print("STD Data Date Range:", std_data['DateTime'].min(), "-", std_data['DateTime'].max())
print("TOU Data Date Range:", tou_data['DateTime'].min(), "-", tou_data['DateTime'].max())

print("STD Data:")
print(std_data.head())
print("\nTOU Data:")
print(tou_data.head())

STD IDs (count: 50): ['MAC000062', 'MAC000006', 'MAC000019', 'MAC000026', 'MAC000037']
TOU IDs (count: 50): ['MAC000124', 'MAC000147', 'MAC000014', 'MAC000015', 'MAC000194']
STD Data Date Range: 2013-01-01 00:00:00 - 2014-02-28 00:00:00
TOU Data Date Range: 2013-01-01 00:00:00 - 2014-02-28 00:00:00
STD Data:
          LCLid stdorToU            DateTime KWH/hh (per half hour) 
3840  MAC000002      Std 2013-01-01 00:00:00                  0.219 
3841  MAC000002      Std 2013-01-01 00:30:00                  0.241 
3842  MAC000002      Std 2013-01-01 01:00:00                  0.191 
3843  MAC000002      Std 2013-01-01 01:30:00                  0.235 
3844  MAC000002      Std 2013-01-01 02:00:00                  0.182 

TOU Data:
               LCLid stdorToU            DateTime KWH/hh (per half hour) 
134158912  MAC000005      ToU 2013-01-01 00:00:00                  0.096 
134158913  MAC000005      ToU 2013-01-01 00:30:00                  0.111 
134158914  MAC000005      ToU 2013-01-01 01

#**Preprocessing**

In [6]:
# Renaming the Column name KWH/hh (per half hour) to Consumption in std Data
std_data.rename(columns={'KWH/hh (per half hour) ': 'Consumption'}, inplace=True)
tou_data.rename(columns={'KWH/hh (per half hour) ': 'Consumption'}, inplace=True)

In [7]:
# Droping Column stdorTou from both Std & ToU dataframes
std_data.drop('stdorToU', axis=1, inplace=True)
tou_data.drop('stdorToU', axis=1, inplace=True)

In [8]:
# Resetting the Index of Dataframes
std_data = std_data.reset_index(drop=True)
tou_data = tou_data.reset_index(drop=True)


In [9]:
std_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015780 entries, 0 to 1015779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   LCLid        1015780 non-null  object        
 1   DateTime     1015780 non-null  datetime64[ns]
 2   Consumption  1015780 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 23.2+ MB


In [10]:
nan_count = std_data.isna().sum()
print(f"Number of NaN values: {nan_count}")


Number of NaN values: LCLid          0
DateTime       0
Consumption    0
dtype: int64


In [11]:
std_data

,LCLid,DateTime,Consumption
0,MAC000002,2013-01-01 00:00:00,0.219
1,MAC000002,2013-01-01 00:30:00,0.241
2,MAC000002,2013-01-01 01:00:00,0.191
3,MAC000002,2013-01-01 01:30:00,0.235
4,MAC000002,2013-01-01 02:00:00,0.182
...,...,...,...
1015775,MAC000068,2014-02-27 22:30:00,0.924
1015776,MAC000068,2014-02-27 23:00:00,0.876
1015777,MAC000068,2014-02-27 23:30:00,0.306
1015778,MAC000068,2014-02-28 00:00:00,0.253


In [12]:
tou_data

,LCLid,DateTime,Consumption
0,MAC000005,2013-01-01 00:00:00,0.096
1,MAC000005,2013-01-01 00:30:00,0.111
2,MAC000005,2013-01-01 01:00:00,0.071
3,MAC000005,2013-01-01 01:30:00,0.035
4,MAC000005,2013-01-01 02:00:00,0.035
...,...,...,...
1015126,MAC000344,2014-02-27 22:30:00,0.188
1015127,MAC000344,2014-02-27 23:00:00,0.142
1015128,MAC000344,2014-02-27 23:30:00,0.125
1015129,MAC000344,2014-02-28 00:00:00,0.151


In [13]:
#Parsing the DateTime column in both data frames
std_data['DateTime'] = pd.to_datetime(std_data['DateTime'])
tou_data['DateTime'] = pd.to_datetime(tou_data['DateTime'])

## **Missing Value Imputation for STD Data**

In [14]:
std_data['DateTime'] = pd.to_datetime(std_data['DateTime'])  # Convert to datetime if not already
std_data['Consumption'] = pd.to_numeric(std_data['Consumption'], errors='coerce')  # Ensure numeric

# Find complete DateTime range
min_date = std_data['DateTime'].min()
max_date = std_data['DateTime'].max()
freq = '30min'  # Frequency of 30 minutes

all_dates = pd.date_range(start=min_date, end=max_date, freq=freq)

In [15]:
# Initialize an empty DataFrame to store missing data and a list to store missing counts
missing_data = pd.DataFrame()
missing_counts = []

# Identify missing DateTime values for each LCLid and count missing days
for lid, group in std_data.groupby('LCLid'):
    existing_dates = pd.to_datetime(group['DateTime'])
    missing_dates = pd.Series(all_dates.difference(existing_dates))  # Convert to Pandas Series

    if not missing_dates.empty:
        # Count unique missing days
        missing_days_count = missing_dates.dt.date.nunique()
        missing_counts.append({'LCLid': lid, 'MissingDaysCount': missing_days_count})

        # Create a DataFrame for the missing rows
        missing_rows = pd.DataFrame({
            'LCLid': lid,
            'MissingDateTime': missing_dates
        })
        missing_data = pd.concat([missing_data, missing_rows], ignore_index=True)

# Convert missing counts to a DataFrame
missing_counts_df = pd.DataFrame(missing_counts)

# Print the missing counts DataFrame
print(missing_counts_df)

# Print a sample of the missing data DataFrame
print(missing_data.head())


        LCLid  MissingDaysCount
0   MAC000002                 1
1   MAC000003                 2
2   MAC000004                 1
3   MAC000007                 5
4   MAC000009                 1
5   MAC000011                 1
6   MAC000012                 2
7   MAC000013                 1
8   MAC000020                 2
9   MAC000021                 1
10  MAC000022                 2
11  MAC000023                 1
12  MAC000024                 1
13  MAC000025                 6
14  MAC000026                 5
15  MAC000027                 2
16  MAC000029                 1
17  MAC000030                 3
18  MAC000032                 1
19  MAC000033                 2
20  MAC000034                 2
21  MAC000035                 2
22  MAC000036                 1
23  MAC000037                 1
24  MAC000038                 3
25  MAC000039                 2
26  MAC000041                 2
27  MAC000042                 2
28  MAC000043                 2
29  MAC000045                 2
30  MAC0

In [16]:
# Extract Hour and Month from MissingDateTime
missing_data['MissingDateTime'] = pd.to_datetime(missing_data['MissingDateTime'])
missing_data['Hour'] = missing_data['MissingDateTime'].dt.hour
missing_data['Month'] = missing_data['MissingDateTime'].dt.month

# Extract Hour and Month from DateTime in original std_data
std_data['Hour'] = std_data['DateTime'].dt.hour
std_data['Month'] = std_data['DateTime'].dt.month

# Calculate the average consumption for each LCLid, Month, and Hour
avg_consumption = std_data.groupby(['LCLid', 'Month', 'Hour'])['Consumption'].mean().reset_index()
avg_consumption.rename(columns={'Consumption': 'AvgConsumption'}, inplace=True)

In [17]:
# Merge missing_data with avg_consumption to get the imputed values
missing_data = pd.merge(missing_data, avg_consumption, how='left', on=['LCLid', 'Month', 'Hour'])

# Rename the AvgConsumption column to Consumption to match original std_data structure
missing_data.rename(columns={'AvgConsumption': 'Consumption'}, inplace=True)

In [18]:
# Check for any remaining missing values
remaining_missing = missing_data[missing_data['Consumption'].isnull()]

if remaining_missing.empty:
    print("No missing values remain after imputation.")
else:
    print("There are still missing values after imputation:")
    print(remaining_missing)

No missing values remain after imputation.


In [19]:
# Call the function and get the imputed DataFrame
std_data_imputed = merge_imputed_data(std_data, missing_data)
print(std_data_imputed)

             LCLid            DateTime  Consumption  Hour  Month
0        MAC000002 2013-01-01 00:00:00        0.219   0.0    1.0
1        MAC000002 2013-01-01 00:30:00        0.241   0.0    1.0
2        MAC000002 2013-01-01 01:00:00        0.191   1.0    1.0
3        MAC000002 2013-01-01 01:30:00        0.235   1.0    1.0
4        MAC000002 2013-01-01 02:00:00        0.182   2.0    1.0
...            ...                 ...          ...   ...    ...
1015945  MAC000068 2014-02-27 22:30:00        0.924  22.0    2.0
1015946  MAC000068 2014-02-27 23:00:00        0.876  23.0    2.0
1015947  MAC000068 2014-02-27 23:30:00        0.306  23.0    2.0
1015948  MAC000068 2014-02-28 00:00:00        0.253   0.0    2.0
1015949  MAC000068 2014-02-28 00:00:00        0.253   0.0    2.0

[1015950 rows x 5 columns]




## **Missing Value imputation for ToU Data**



In [20]:
tou_data['DateTime'] = pd.to_datetime(tou_data['DateTime'])  # Convert to datetime if not already
tou_data['Consumption'] = pd.to_numeric(tou_data['Consumption'], errors='coerce')  # Ensure numeric

# Find complete DateTime range
min_date = tou_data['DateTime'].min()
max_date = tou_data['DateTime'].max()
freq = '30min'  # Frequency of 30 minutes

all_dates = pd.date_range(start=min_date, end=max_date, freq=freq)

In [21]:
# Initialize an empty DataFrame to store missing data and a list to store missing counts
missing_data = pd.DataFrame()
missing_counts = []

# Identify missing DateTime values for each LCLid and count missing days
for lid, group in tou_data.groupby('LCLid'):
    existing_dates = pd.to_datetime(group['DateTime'])
    missing_dates = pd.Series(all_dates.difference(existing_dates))  # Convert to Pandas Series

    if not missing_dates.empty:
        # Count unique missing days
        missing_days_count = missing_dates.dt.date.nunique()
        missing_counts.append({'LCLid': lid, 'MissingDaysCount': missing_days_count})

        # Create a DataFrame for the missing rows
        missing_rows = pd.DataFrame({
            'LCLid': lid,
            'MissingDateTime': missing_dates
        })
        missing_data = pd.concat([missing_data, missing_rows], ignore_index=True)

# Convert missing counts to a DataFrame
missing_counts_df = pd.DataFrame(missing_counts)

# Print the missing counts DataFrame
print(missing_counts_df)

# Print a sample of the missing data DataFrame
print(missing_data.head())


        LCLid  MissingDaysCount
0   MAC000005                 1
1   MAC000014                 7
2   MAC000015                11
3   MAC000017                 2
4   MAC000031                 4
5   MAC000044                 1
6   MAC000046                 1
7   MAC000051                 8
8   MAC000052                 2
9   MAC000064                 1
10  MAC000075                 1
11  MAC000076                 5
12  MAC000082                 1
13  MAC000088                 1
14  MAC000107                 1
15  MAC000109                 2
16  MAC000124                 4
17  MAC000141                 1
18  MAC000158                 6
19  MAC000170                 5
20  MAC000173                 5
21  MAC000186                 5
22  MAC000194                 6
23  MAC000195                 2
24  MAC000198                 1
25  MAC000219                 5
26  MAC000236                 4
27  MAC000247                 4
28  MAC000257                 3
29  MAC000259                 6
30  MAC0

In [22]:
# Extract Hour and Month from MissingDateTime
missing_data['MissingDateTime'] = pd.to_datetime(missing_data['MissingDateTime'])
missing_data['Hour'] = missing_data['MissingDateTime'].dt.hour
missing_data['Month'] = missing_data['MissingDateTime'].dt.month

# Extract Hour and Month from DateTime in original std_data
tou_data['Hour'] = std_data['DateTime'].dt.hour
tou_data['Month'] = std_data['DateTime'].dt.month

# Calculate the average consumption for each LCLid, Month, and Hour
avg_consumption = tou_data.groupby(['LCLid', 'Month', 'Hour'])['Consumption'].mean().reset_index()
avg_consumption.rename(columns={'Consumption': 'AvgConsumption'}, inplace=True)

In [23]:
# Merge missing_data with avg_consumption to get the imputed values
missing_data = pd.merge(missing_data, avg_consumption, how='left', on=['LCLid', 'Month', 'Hour'])

# Rename the AvgConsumption column to Consumption to match original std_data structure
missing_data.rename(columns={'AvgConsumption': 'Consumption'}, inplace=True)

In [24]:
# Check for any remaining missing values
remaining_missing = missing_data[missing_data['Consumption'].isnull()]

if remaining_missing.empty:
    print("No missing values remain after imputation.")
else:
    print("There are still missing values after imputation:")
    print(remaining_missing)

No missing values remain after imputation.


In [25]:
# Call the function and get the imputed DataFrame
tou_data_imputed = merge_imputed_data(tou_data, missing_data)
print(tou_data_imputed)

             LCLid            DateTime  Consumption  Hour  Month
0        MAC000005 2013-01-01 00:00:00        0.096   0.0    1.0
1        MAC000005 2013-01-01 00:30:00        0.111   0.0    1.0
2        MAC000005 2013-01-01 01:00:00        0.071   1.0    1.0
3        MAC000005 2013-01-01 01:30:00        0.035   1.0    1.0
4        MAC000005 2013-01-01 02:00:00        0.035   2.0    1.0
...            ...                 ...          ...   ...    ...
1015945  MAC000344 2014-02-27 22:30:00        0.188  10.0    2.0
1015946  MAC000344 2014-02-27 23:00:00        0.142  10.0    2.0
1015947  MAC000344 2014-02-27 23:30:00        0.125  11.0    2.0
1015948  MAC000344 2014-02-28 00:00:00        0.151  11.0    2.0
1015949  MAC000344 2014-02-28 00:00:00        0.151  12.0    2.0

[1015950 rows x 5 columns]


## **EDA**